## Kinematics

The kinematics of a robot used in the thesis are introduced in this section along with some basic concepts and algorithmic implementations. The first subsection discusses the conversions between coordinate frames and the selected convention. Next, a brief explanation of the Geometric Jacobian computation procedure and forward kinematics is given.

### Denavit-Hartenberg Convention

Although reference frames attached to the links can be chosen arbitrarily, it is practical to select them systematically. A widely used convention for describing the 
kinematic geometry of robotic arms is the Denavit-Hartenberg (DH) convention <cite id="gx4bn"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite><cite id="10z4u"><a href="#zotero%7C17740253%2F3J5DS9LQ">(Denavit &#38; Hartenberg, 2021)</a></cite>

The homogeneous transformation between frame $i$ and $i-1$ is obtained by multiplying the rotation $\mathbf{T}_{R_z}$ around the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_z}$ along the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_x}$ along the $\hat{\mathbf{x}}_i$-axis, and the rotation $\mathbf{T}_{R_x}$ around the $\hat{\mathbf{x}}_i$-axis, as follows:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = \mathbf{T}_{R_z}(\theta_i) \mathbf{T}_{T_z}(d_i) \mathbf{T}_{T_x}(a_i) \mathbf{T}_{R_x}(\alpha_i)
\end{equation}

Combining these, the homogeneous transformation matrix is:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = 
\begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} c_{\alpha_i} & s_{\theta_i} s_{\alpha_i} & a_i c_{\theta_i} \\
s_{\theta_i} & c_{\theta_i} c_{\alpha_i} & -c_{\theta_i} s_{\alpha_i} & a_i s_{\theta_i} \\
0 & s_{\alpha_i} & c_{\alpha_i} & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
\end{equation}

In this expression, $c_{\theta_i}$ and $s_{\theta_i}$ represent $\cos(\theta_i)$ and $\sin(\theta_i)$, respectively. Notably, the homogeneous transformation ${}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i)$ depends on a single variable once the reference frame is established and the other parameters are fixed. This variable is determined by the type of joint: $d_i$ for prismatic joints and $\theta_i$ for revolute joints. A more elaborate theory can be found in <cite id="zhen5"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite>.

In [1]:
def dh_transform(a, alpha, d, theta):
    """
    Compute Denavit-Hartenberg transformation matrix.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha),  np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta),  np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0,             np.sin(alpha),                 np.cos(alpha),                 d],
        [0,             0,                             0,                             1]
    ])


Although reference frames attached to the links can be chosen arbitrarily, it is practical to select them systematically. A widely used convention for describing the position and orientation of successive link frames relative to each other is the Denavit-Hartenberg (DH) convention[DH55-jens] <cite id="gx4bn"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite><cite id="10z4u"><a href="#zotero%7C17740253%2F3J5DS9LQ">(Denavit &#38; Hartenberg, 2021)</a></cite>


#### define the parmaetrs 

The homogeneous transformation between frame $i$ and $i-1$ is obtained by multiplying the rotation $\mathbf{T}_{R_z}$ around the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_z}$ along the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_x}$ along the $\hat{\mathbf{x}}_i$-axis, and the rotation $\mathbf{T}_{R_x}$ around the $\hat{\mathbf{x}}_i$-axis, as follows:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = \mathbf{T}_{R_z}(\theta_i) \mathbf{T}_{T_z}(d_i) \mathbf{T}_{T_x}(a_i) \mathbf{T}_{R_x}(\alpha_i)
\label{eq:3.14} \tag{3.14}
\end{equation}

This sequence of transformations can be represented by the following matrix multiplications:

$$
\mathbf{T}_{R_z}(\theta_i) = \begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} & 0 & 0 \\
s_{\theta_i} & c_{\theta_i} & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

$$
\mathbf{T}_{T_z}(d_i) = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

$$
\mathbf{T}_{T_x}(a_i) = \begin{bmatrix}
1 & 0 & 0 & a_i \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

$$
\mathbf{T}_{R_x}(\alpha_i) = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & c_{\alpha_i} & -s_{\alpha_i} & 0 \\
0 & s_{\alpha_i} & c_{\alpha_i} & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

Combining these, the homogeneous transformation matrix is:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = 
\begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} c_{\alpha_i} & s_{\theta_i} s_{\alpha_i} & a_i c_{\theta_i} \\
s_{\theta_i} & c_{\theta_i} c_{\alpha_i} & -c_{\theta_i} s_{\alpha_i} & a_i s_{\theta_i} \\
0 & s_{\alpha_i} & c_{\alpha_i} & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
\label{eq:3.15} \tag{3.15}
\end{equation}

In this expression, $c_{\theta_i}$ and $s_{\theta_i}$ represent $\cos(\theta_i)$ and $\sin(\theta_i)$, respectively. Notably, the homogeneous transformation ${}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i)$ depends on a single variable once the reference frame is established and the other parameters are fixed. This variable is determined by the type of joint: $d_i$ for prismatic joints and $\theta_i$ for revolute joints. A more elaborate theory can be found in <cite id="zhen5"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite>.

In [2]:
def dh_transform(a, alpha, d, theta):
    """
    Compute Denavit-Hartenberg transformation matrix.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha),  np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta),  np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0,             np.sin(alpha),                 np.cos(alpha),                 d],
        [0,             0,                             0,                             1]
    ])

### Forward Kinematics

The kinematic representation of a robot manipulator consists of multiple links connected by joints. The links are generally rigid bodies that form the robot’s structure, while the joints are the movable components that connect these links. Joints can be either active, with an actuator, or passive, without an actuator. Additionally, joints are commonly found in serial robots are usually prismatic or revolute. Each type of such joint adds one degree of freedom (dof) to the system: the joint position $q_i$ for prismatic joints or the joint angle $q_i$ for revolute joints. The dof of an active joint can be controlled individually by a microcontroller, which relies on feedback from an encoder measuring the joint's position. The entire kinematic structure is typically referred to as a chain, which can be of two types:

* Open chain: This configuration has only one path in the structure that connects the two extremes, commonly referred to as the base and the end-effector.
* Closed chain: This configuration includes loops within the kinematic structure.


In this thesis, only active revolute joints arranged in an open chain are considered.

A common problem in robotics is determining the Cartesian position $\mathbf{x} \in \mathbb{R}^3$ of a specific point on a link given the joint angles $\mathbf{q} \in \mathbb{R}^n$. This problem is known as forward kinematics, which can be efficiently solved using homogeneous transformations. For instance, the position of the origin of the end-effector’s reference frame $\text{RF}_e$ in the base frame $\text{RF}_b$ can be determined using the following equation:

$$
{}^b\mathbf{T}_e(\mathbf{q}) = {}^b\mathbf{T}_0 \, {}^0\mathbf{T}_1(q_1) \cdots {}^{n-1}\mathbf{T}_e(q_n) \, {}^n\mathbf{T}_e,
$$

where ${}^b\mathbf{T}_0$ and ${}^n\mathbf{T}_e$ are two constant homogeneous transformations needed to convert a point from frame 0 to the base frame and from the end-effector frame to frame $n$, respectively. For a more detailed description, refer to the relevant section in
 <cite id="wc30q"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite>.

In [3]:
def compute_forward_kinematics(joint_angles, dh_params):
    """
    Compute the forward kinematics for a robot given DH parameters and joint angles.

    :param dh_params: List of DH parameters [(a, alpha, d, theta), ...]
    :param joint_angles: List of joint angles [theta1, theta2, ..., thetaN]
    :return: List of transformation matrices from base to each joint and the final transformation matrix T_0_n
    """
    num_joints = len(joint_angles)
    T = np.eye(4)
    transformations = [T.copy()]
    
    # Compute forward kinematics to get transformations
    for i, (a, alpha, d, theta) in enumerate(dh_params):
        theta += joint_angles[i]  # Update theta with joint angle
        T_i = dh_transform(a, alpha, d, theta)
        T = np.dot(T, T_i)
        transformations.append(T.copy())
    
    T_0_n = transformations[-1]  # Final transformation matrix to the TCP (end-effector)
    return transformations, T_0_n

###  Geometric Jacobian

The differential kinematic gives the relationship between the joint velocities q̇ and
the task space instantaneous velocities [ ṗ T , ω T ] T . where the vector ṗ = [ ẋ, ẏ, ż] T
describes the Cartesian linear velocities, and ω = [ω x , ω y , ω z ] T are the angular
velocities in the base frame. The matrix that maps these two quantities is called
geometric Jacobian J (q) ∈ R 6×n
$$
\begin{bmatrix}
\dot{\mathbf{p}} \\
\boldsymbol{\omega}
\end{bmatrix}
= \mathbf{J}(\mathbf{q}) \dot{\mathbf{q}}.
$$
The reason why it is called geometric Jacobian is that the derivation is done in a
geometric way and not directly via a time differentiation of the forward kinematics.
This last Jacobian is called analytic Jacobian. Throughout this thesis, Jacobians are
always meant to be the geometric Jacobians.
It is helpful for the derivation to partition the Jacobian into 3 × 1 sub-vectors called
j p,i and j o,i , where j p,i describes the contribution of joint i to the linear velocity
and j o,i (q) describes the contribution of joint i to the angular velocity. With the
partitioned Jacobian equation (2.6) is re-written as
$$
\begin{pmatrix}
\dot{\mathbf{p}} \\
\boldsymbol{\omega}
\end{pmatrix}
= \begin{pmatrix}
j_{p,1} & \cdots & j_{p,n} \\
j_{o,1} & \cdots & j_{o,n}
\end{pmatrix}
\dot{\mathbf{q}}.
$$
In the following, only revolute joints are considered for the Jacobian computation.
An exaustive explanation can be found in [18]. It is convenient to proceed separately
for the computation of the linear and angular velocity.
#### Linear Velocity
The linear velocity of the desired frame RF e due to the angular velocity of a
revolute joint i can be calculated as follows
$$
J_{p,i} \dot{q}_i = \boldsymbol{\omega}_{i-1,i} \times \mathbf{r}_{i-1,e} \\
= \dot{q}_i \mathbf{z}_{i-1} \times (\mathbf{p}_{0,e} - \mathbf{p}_{0,i-1}).
$$
The vector z i−1 describes the rotation axis of joint i, the vectors p 0,e and p 0,i−1 are
the position of frame RF e and i − 1 relative to the base frame and represented in frame i. The joint velocity of revolute joint i is denoted by q̇ i and
$$
\mathbf{J}_{p,i} = \mathbf{z}_{i-1} \times (\mathbf{p}_{0,s} - \mathbf{p}_{0,i-1}),
$$
holds.
#### Angular Velocity
The expression to calculate the relative angular velocity between two consecutive elements in the robot chain is given by
\begin{equation}
\boldsymbol{\omega}_{i-1,i} = \dot{q}_i \mathbf{z}_{i-1}
\end{equation}
where $\boldsymbol{\omega}_{i-1,i}$ describes the relative angular velocity from frame $i-1$ to frame $i$. For a revolute joint, the Jacobian that relates the angular velocity in the joint space to the angular velocity in the Cartesian space is given by
\begin{equation}
\mathbf{J}_{o,i} \dot{q}_i = \dot{q}_i \mathbf{z}_{i-1}
\end{equation}
and then
\begin{equation}
\mathbf{J}_{o,i} = \mathbf{z}_{i-1}
\end{equation}
where $\mathbf{z}_{i-1}$ is the third column of ${}^0\mathbf{R}_{i-1}$, hence the rotation axis of joint $i$, that is computed by
\begin{equation}
\mathbf{z}_{i-1} = {}^0\mathbf{R}_1(q_1) \cdots {}^{i-2}\mathbf{R}_{i-1}(q_{i-1}) \mathbf{z}_0.
\end{equation}
Here, $\mathbf{z}_0 = [0, 0, 1]^\mathrm{T}$ allows the selection of the third column.



In [4]:
def compute_geometric_jacobian(transformations):
    """
    Compute the geometric Jacobian for a robot given the transformation matrices.

    :param transformations: List of transformation matrices from the base to each joint
    :return: Geometric Jacobian matrix (6 x N) in the base frame
    """
    num_joints = len(transformations) - 1  # Excluding the base frame
    jacobian_base = np.zeros((6, num_joints))
    
    # End-effector position
    T_0_n = transformations[-1]
    o_n = T_0_n[:3, 3]

    # Compute each column of the Jacobian in the base frame
    for i in range(num_joints):
        T_0_i = transformations[i]
        o_i = T_0_i[:3, 3]
        z_i = T_0_i[:3, 2]

        jacobian_base[:3, i] = np.cross(z_i, (o_n - o_i))  # Linear part
        jacobian_base[3:, i] = z_i  # Angular part

    return jacobian_base

In [5]:
def convert_jacobian_to_tcp(jacobian_base, T_0_n):
    """
    Convert a Jacobian from the base frame to the TCP frame.
    
    :param jacobian_base: Jacobian matrix in the base frame (6 x 6)
    :param T_0_n: Transformation matrix from the base to the TCP (4 x 4)
    :return: Jacobian matrix (6 x 6) in the TCP frame
    """
    R_0_n = T_0_n[:3, :3]
    T_base_to_tcp = np.block([
        [R_0_n.T, np.zeros((3, 3))],
        [np.zeros((3, 3)), R_0_n.T]
    ])
    
    jacobian_tcp = np.dot(T_base_to_tcp, jacobian_base)
    return jacobian_tcp